In [74]:
# 系统包
import os
import sys

In [75]:
# 机器学习与相关工具包
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import time
from datetime import timedelta
import random
from sklearn import metrics
import csv
from collections import defaultdict
import torch.nn.functional as F
import logging

In [76]:
#调用LTP工具
from ltp import LTP
ltp = LTP('base')

In [77]:
# 调用transformers包
from transformers.models.bert.modeling_bert import BertModel, BertConfig
from transformers import BertTokenizer
from transformers import AdamW, get_constant_schedule_with_warmup

In [78]:
logger = logging.getLogger(__name__)

In [79]:
def read_conjs(path = '../Data/Dataset/conjs.txt'):
    ans = []
    with open(path,'r',encoding = 'utf-8-sig') as f:
        lines = f.readlines()
        for i, line in tqdm(enumerate(lines)):
            line = line.strip()
            ans.append(line)
    return ans

In [129]:
def get_time_dif(start_time):
    """
    获取已使用时间
    :param: start_time是任务开始的时间
    :return: 返回整体训练的时间，单位s
    """
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [130]:
def pos_seq(tokens, pos, conjs):
    '''
        创建pos掩码序列
    '''
    ans = []
    for i,token in enumerate(tokens):
        if pos[i] == 'c' and tokens[i] in conjs:
            md = [1 for c in token]
        else:
            md = [0 for c in token]
        ans.extend(md)
    return [0] + ans

In [533]:
def load_data(path, tokenizer, pad_size = 150):
    '''
    读取数据集
    :param: path数据集路径
    :param: tokenizer 分词器
    :param: pad_size: 数据的填充长度
    :return: (token_ids, label, seq_len, mask) Type: List
    '''
    outputs = defaultdict(list)
    checks = []
    
    max_len = pad_size - 1
    
    ans = read_conjs()
    
    with open(path,'r',encoding = 'utf-8-sig') as f:
        lines = csv.reader(f)
        for i, line in tqdm(enumerate(lines)):
            if i == 0:
                continue
            
            context, label = line
            context = context.strip()
            seg, hidden = ltp.seg([context])
            pos = ltp.pos(hidden)[0]
            seg = seg[0]
            
            pos = pos_seq(seg, pos, ans)
            
            label = int(label)
            assert label in {0, 1}
            
            tokens = tokenizer.tokenize(context)
            tokens = ['[CLS]'] + tokens
            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            
            seq_len = len(tokens)
            mask = []
            
            if seq_len >= max_len:
                input_ids = input_ids[:max_len] + tokenizer.convert_tokens_to_ids(['[SEP]'])
                mask = [1] * pad_size
                
            elif seq_len < max_len:
                input_ids += tokenizer.convert_tokens_to_ids(['[SEP]']) + [0] * (pad_size - seq_len - 1)
                mask = [1] * (seq_len + 1) + [0] * (pad_size - seq_len - 1)
            
            pos_len = len(pos)
            if pos_len >= pad_size:
                pos = pos[:pad_size]
            else:
                pos += [0]*(pad_size - pos_len)
            
            
            outputs['input_ids'].append(input_ids)
            outputs['attention_mask'].append(mask)
            outputs['token_type_ids'].append([0] * pad_size)
            outputs['pos_ids'].append(pos)
            outputs['labels'].append(label)
    
    return outputs

In [534]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
testset_path = '/Users/chenshaojie/Downloads/Graduation Project/Data/Dataset/Dataset/test_preprocess.csv'

In [535]:
outputs = load_data(testset_path,tokenizer)

160it [00:00, 416825.24it/s]
592it [04:21,  2.27it/s]


In [536]:
class ReDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
        return item
    
    def __len__(self):
        
        return len(self.data['input_ids'])

In [537]:
testset = ReDataset(outputs)
dataloader = DataLoader(testset, batch_size = 3)

In [538]:
len(testset[43]['pos_ids'])

150

In [539]:
for batch in dataloader:
    pass

In [540]:
batch

{'input_ids': tensor([[ 101, 2824, 5885, 7566, 2193, 2671, 6381, 1568, 8024, 1377, 2667,  679,
          5543, 6656, 1920, 2157, 1398, 1343, 1266, 2785, 3777, 2861, 5298,  749,
          4867, 1920, 2157, 1391, 1962, 1600, 1962, 4381, 1962, 5314,  712, 5356,
          4867, 2195,  749, 8013, 2523, 1962, 1391, 4638, 8024, 5445,  684, 7623,
          1072,  738, 4023,  778,  511,  511, 5401, 7608, 5401, 1690, 8013, 8013,
          8013, 4696, 1395, 1164, 6821, 3189, 2094, 1611, 1611,  172,  712, 5356,
          4495, 3189, 2571,  727,  172,  172,  102,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
   

In [410]:
hidden_state = torch.randn(3,150,768)

In [446]:
pos = pos_mask(batch['pos_ids'],hidden_state)
pos = neighbour_pool(pos)

In [447]:
output = attention_net(pos, hidden_state)

In [449]:
pool = GlobalMaxPool1d()

In [445]:
def attention_net(pos, hidden_states):
    outputs = []
    ans = []
    for idx, p in enumerate(pos):
        hidden_state = hidden_states[idx] 
        for e in p:
            
            attn_hidden_layer = torch.tanh(torch.matmul(hidden_state, e))
            exps = torch.exp(attn_hidden_layer)
            alpha = exps / torch.sum(exps)
            attn_output = torch.sum(hidden_state * alpha.unsqueeze(-1), dim = 0)
            ans.append(attn_output.unsqueeze(-1))
      
        if len(ans) > 1:
            attn_outputs = torch.cat(ans[:], dim = -1)
            attn_outputs = pool(attn_outputs.unsqueeze(0))
            outputs.append(attn_outputs.squeeze(-1))
        else:
            attn_outputs = ans[0].squeeze(-1)
            outputs.append(attn_outputs.unsqueeze(0))
        ans = []
    return outputs

In [426]:
def pos_mask(pos, hidden_states):
    '''
        利用pos信息提取hidden_state中的连词
    '''
    pos = pos.unsqueeze(-1)
    pos = pos.expand(pos.shape[0],pos.shape[1],hidden_states.shape[-1])
    
    out = pos * hidden_states
    return out
    
def neighbour_pool(hidden_states):
    '''
        将相邻连词中的字进行max_pool
    '''
    check = torch.zeros(hidden_states.shape[-1])
    outputs = []
    output = []
    ans = []
    for idx, hidden_state in enumerate(hidden_states):
        for state in hidden_state:
            if not state.equal(check):
                ans.append(state.unsqueeze(-1))
            elif ans:
                if len(ans) > 1:
                    
                    cat_tensor = torch.cat(ans[:],dim = -1)
                    cat_tensor = pool(cat_tensor.unsqueeze(0))
                    cat_tensor = cat_tensor.squeeze()
                else:
                    cat_tensor = ans[0].squeeze(-1)
                output.append(cat_tensor)
                ans = []
        if not output:
            outputs.append([torch.zeros(hidden_state.shape[-1])])
        else:
            outputs.append(output[:])
        output = []

    return outputs

In [459]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d,self).__init__()
    def forward(self, x):
        return torch.max_pool1d(x,kernel_size=x.shape[-1])

In [500]:
class CISTM(nn.Module):
    def __init__(self, bert_path, num_labels, device):
        super(CISTM,self).__init__()
        
        self.device = device
        self.bert = BertModel.from_pretrained(bert_path)
        for _,param in self.bert.named_parameters():
            param.requires_grad = True
        self.linear = nn.Linear(self.bert.config.hidden_size,num_labels)
        self.max_pool = GlobalMaxPool1d()
        self.attn_linear = nn.Linear(self.bert.config.hidden_size,self.bert.config.hidden_size)
    
    def pos_mask(self, pos, hidden_states):
        '''
            利用pos信息提取hidden_state中的连词
        '''
        pos = pos.unsqueeze(-1)
        pos = pos.expand(pos.shape[0],pos.shape[1],hidden_states.shape[-1])
        
        out = pos * hidden_states
        return out
    
    def neighbour_pool(self, hidden_states):
        '''
            将相邻连词中的字进行max_pool
        '''
        check = torch.zeros(hidden_states.shape[-1])
        outputs = []
        output = []
        ans = []
        for idx, hidden_state in enumerate(hidden_states):
            for state in hidden_state:
                if not state.equal(check):
                    ans.append(state.unsqueeze(-1))
                elif ans:
                    if len(ans) > 1:
                        cat_tensor = torch.cat(ans[:],dim = -1)
                        cat_tensor = self.max_pool(cat_tensor.unsqueeze(0))
                        cat_tensor = cat_tensor.squeeze()
                    else:
                        cat_tensor = ans[0].squeeze(-1)
                    output.append(cat_tensor)
                    ans = []
                
            if not output:
                outputs.append([torch.zeros(hidden_state.shape[-1])])
            else:
                outputs.append(output[:])
            output = []
        return outputs
        
    def pos_attention(self, pos, hidden_states):
        outputs = []
        ans = []
        for idx, p in enumerate(pos):
            hidden_state = hidden_states[idx] 
            for e in p:

                attn_hidden_layer = torch.tanh(torch.matmul(hidden_state, e))
                exps = torch.exp(attn_hidden_layer)
                alpha = exps / torch.sum(exps)
                attn_output = torch.sum(hidden_state * alpha.unsqueeze(-1), dim = 0)
                ans.append(attn_output.unsqueeze(-1))

            if len(ans) > 1:
                attn_outputs = torch.cat(ans[:], dim = -1)
                attn_outputs = self.max_pool(attn_outputs.unsqueeze(0))
                outputs.append(attn_outputs.squeeze(-1))
            else:
                attn_outputs = ans[0].squeeze(-1)
                outputs.append(attn_outputs.unsqueeze(0))
            ans = []
        return outputs
           
        
    def forward(self, inputs, pos_ids):
        outputs = self.bert(**inputs)
        pooler_output = outputs.pooler_output
        hidden_states = outputs.last_hidden_state
            
        pos = self.pos_mask(pos_ids, hidden_states)
        pos = self.neighbour_pool(pos)
        attn_output = self.pos_attention(pos, hidden_states)
        
        attn_output = torch.cat(attn_output[:],dim = 0)
        out = pooler_output + attn_output
        
        out = self.linear(out)
        return out

In [501]:
for batch in dataloader:
    break

In [502]:
inputs = {k:v for k,v in batch.items() if k != 'labels' and k != 'pos_ids'}

In [503]:
model = CISTM('bert-base-chinese',2,torch.device('cpu'))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [504]:
output = model(inputs,batch['pos_ids'])

In [505]:
output.shape

torch.Size([3, 2])

In [46]:
def evaluate(model, data_iter):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for i, batch in enumerate(data_iter):
            inputs = {k:v.to(device) for k,v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predict = torch.max(outputs,dim = 1)[1].cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predict)
    
    f1 = metrics.f1_score(labels_all, predict_all, labels = [0,1], average = 'macro')
    acc = metrics.accuracy_score(labels_all, predict_all)
    return acc, loss_total / len(data_iter),f1      

In [66]:
def train(model, train_iter, test_iter, learning_rate, num_epochs, require_improvement, save_path, device):
    start_time = time.time()
    model.train()
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
    
    optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr = learning_rate)
    
    total_batch = 0
    best_acc = 0
    last_improve = 0
    flag = False
    
    num_correct = 0
    num_loss = 0
    num_train = 0
    
    for epoch in range(num_epochs):
        print('Epoch [{}/{}]'.format(epoch + 1, num_epochs))
        for i,(batch) in enumerate(train_iter):
            inputs = {k:v.to(device) for k,v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(inputs)
            
            loss = F.cross_entropy(outputs, labels)
            
            num_train += len(labels)
            num_correct += torch.sum((torch.max(outputs,dim = 1)[1] == labels).type(torch.int)).item()
            num_loss += loss
             
            loss.backward()
            optimizer.step()
            total_batch += 1
            
            if total_batch % 10 == 0:
                
                dev_acc, dev_f1 = evaluate(model, test_iter)
                if dev_acc > best_acc:
                    best_acc = dev_acc
                    model_to_save =model.module if hasattr(model,'module') else model
                    model_to_save.save_pretrained(save_path)
                    logger.info('Saving model checkpoint to %s',output_dir)
                    
                    improve = '*'
                    last_improve = total_batch
                else:
                    improve = ''
                
                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6},  Train Loss: {3:>5.2},  Train Acc: {2:>6.2%},  Val Loss: {3:>5.2},  Val Acc: {4:>6.4%}, Val F1:{5:>6.4%} Time: {6} {7}'
                
                train_acc, train_loss = nums_correct/num_train, loss/num_train
                logger.info(msg.format(total_batch, train_loss, train_acc, dev_loss, dev_acc, dev_f1,time_dif, improve))
                
                model.train()
            
            if total_batch - last_improve > require_improvement:
                # 验证集loss超过1000batch没下降，结束训练
                logging.info("No optimization for a long time, auto-stopping...")
                flag = True
                break
            
        if flag:
            break

In [ ]:
if __name__ == '__main__':
    torch.cuda.empty_cache()#保证nvidia-smi中的可以显示被释放的显存
    np.random.seed(1)
    torch.manual_seed(1)
    torch.cuda.manual_seed_all(1)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm-ext')

    
    start_time = time.time()
    print('Loading data...')
    trainset_path = '/Users/chenshaojie/Downloads/Graduation Project/Data/Dataset/Dataset/train_preprocess.csv'
    testset_path = '/Users/chenshaojie/Downloads/Graduation Project/Data/Dataset/Dataset/test_preprocess.csv'
    train_set = load_data(trainset_path, tokenizer, pad_size = 150)
    test_set = load_data(testset_path, tokenizer, pad_size = 150)
    
    trainset = ReDataset(train_set)
    testset = ReDataset(test_set)
    
    train_iter = DataLoader(trainset, batch_size = 16, shuffle = True)
    test_iter = DataLoader(testset, batch_size = 16, shuffle = True)
    
    time_dif = get_time_dif(start_time)
    print('Time usage:', time_dif)
    
    model = BERTClassifer('hfl/chinese-bert-wwm-ext', 2)
    train(model, train_iter, test_iter, 3e-5, 1, 10, './', device)

Loading data...
Time usage: 0:00:05


Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1/1]
